### 1. 필요한 라이브러리 및 함수 정의

In [9]:
import os
import numpy as np
import open3d as o3d

# .bin 파일 읽기 함수
def load_bin_file(file_path):
    point_cloud = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
    return point_cloud

# Calibration 파일에서 필요한 변환 행렬을 읽는 함수
def load_calibration(calib_path):
    with open(calib_path, 'r') as f:
        lines = f.readlines()
    
    def extract_matrix(line):
        return np.array(line.strip().split()[1:], dtype=np.float32).reshape(3, 4)
    
    Tr_velo_to_cam = extract_matrix(lines[5])  # Assuming line 6 contains Tr_velo_to_cam
    R0_rect = np.array(lines[4].strip().split()[1:], dtype=np.float32).reshape(3, 3)  # Assuming line 5 contains R0_rect
    
    Tr_velo_to_cam = np.vstack((Tr_velo_to_cam, [0, 0, 0, 1]))
    return Tr_velo_to_cam, R0_rect

# 변환 행렬의 역변환 계산 함수
def inverse_transformation(Tr):
    Tr_inv = np.zeros_like(Tr)
    Tr_inv[:3, :3] = Tr[:3, :3].T
    Tr_inv[:3, 3] = -Tr[:3, :3].T @ Tr[:3, 3]
    Tr_inv[3, 3] = 1
    return Tr_inv

# label_2 파일에서 3D bounding box를 파싱하는 함수
def parse_label_file(label_path):
    boxes = []
    with open(label_path, 'r') as f:
        for line in f:
            elements = line.strip().split()
            obj_type = elements[0]
            if obj_type == 'DontCare':
                continue

            height, length, width = map(float, elements[8:11])
            x, y, z = map(float, elements[11:14])
            rotation_y = float(elements[14])
            
            boxes.append((obj_type, x, y, z, length, width, height, rotation_y))
    return boxes

# 특정 bounding box를 제외한 포인트 필터링 함수
def filter_points_outside_boxes(points, boxes, Tr_cam_to_velo, R0_rect_4x4, scale_factor=1.8):
    mask = np.ones(points.shape[0], dtype=bool)
    
    for box in boxes:
        _, x, y, z, length, width, height, rotation_y = box
        
        length *= scale_factor
        width *= scale_factor
        height *= scale_factor
        
        center_cam = np.array([x, y, z, 1])
        center_lidar = Tr_cam_to_velo @ R0_rect_4x4 @ center_cam
        
        rotation_matrix = np.array([
            [np.cos(rotation_y), -np.sin(rotation_y)],
            [np.sin(rotation_y), np.cos(rotation_y)]
        ])
        
        points_xy = points[:, :2] - np.array([center_lidar[0], center_lidar[1]])
        points_xy_rotated = points_xy @ rotation_matrix.T
        
        mask_x = np.abs(points_xy_rotated[:, 0]) <= length / 2
        mask_y = np.abs(points_xy_rotated[:, 1]) <= width / 2
        mask_z = (points[:, 2] >= center_lidar[2]) & (points[:, 2] <= center_lidar[2] + height)
        
        mask &= ~(mask_x & mask_y & mask_z)
    
    filtered_points = points[mask]
    return filtered_points

# .bin 파일 저장 함수
def save_bin_file(file_path, points):
    points.astype(np.float32).tofile(file_path)
    print(f"Saved file: {file_path}")


### 2. 파일 처리 및 디렉토리 구조 생성

In [10]:
def process_and_save_filtered_data(file_name, velo_dir, label_dir, calib_dir, output_dir, scale_factor=1.8):
    # 파일 경로 설정
    velo_file = os.path.join(velo_dir, f"{file_name}.bin")
    label_file = os.path.join(label_dir, f"{file_name}.txt")
    calib_file = os.path.join(calib_dir, f"{file_name}.txt")

    # 데이터 로드
    points = load_bin_file(velo_file)
    boxes = parse_label_file(label_file)
    Tr_velo_to_cam, R0_rect = load_calibration(calib_file)
    Tr_cam_to_velo = inverse_transformation(Tr_velo_to_cam)
    R0_rect_4x4 = np.vstack((np.hstack((R0_rect, np.zeros((3, 1)))), [0, 0, 0, 1]))

    # 출력 디렉토리 생성
    file_output_dir = os.path.join(output_dir, file_name)
    if not os.path.exists(file_output_dir):
        os.makedirs(file_output_dir)

    # 각 bounding box 제거 후 데이터 저장
    for i, box in enumerate(boxes):
        filtered_points = filter_points_outside_boxes(points, [box], Tr_cam_to_velo, R0_rect_4x4, scale_factor=scale_factor)
        output_file_path = os.path.join(file_output_dir, f"{file_name}-{i + 1}.bin")
        save_bin_file(output_file_path, filtered_points)


### 3. 메인코드 실행

object파일 실행시

In [ ]:

# 주요 경로 설정
output_dir = '/media/ssd3/lab/jojeon/KITTI/data_spoofing/velodyne/training_delete'
velo_dir = '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_velodyne/training/velodyne'
label_dir = '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_label_2/training/label_2'
calib_dir = '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_calib/training/calib'


# 파일 이름 자동 탐색 함수
def get_all_file_names(velo_dir):
    """
    지정된 디렉토리에서 .bin 파일 이름(확장자 제외)만 가져옴.
    """
    file_names = [
        os.path.splitext(f)[0]  # 확장자 제거
        for f in os.listdir(velo_dir)
        if f.endswith('.bin')  # .bin 파일만 필터링
    ]
    return sorted(file_names)  # 파일 이름 정렬 (예: 000000, 000001 순서)

# 모든 .bin 파일 이름 가져오기
file_names = get_all_file_names(velo_dir)

# 각 파일 처리
for file_name in file_names:
    process_and_save_filtered_data(file_name, velo_dir, label_dir, calib_dir, output_dir, scale_factor=2.0)


FileNotFoundError: [Errno 2] No such file or directory: '/media/ssd3/lab/sypark/KITTI/object/2d_object/data_object_label_2/testing/label_2/000000.txt'

tracking파일 이용시 사용해야함